# Applied Criptography - Laboratory 5

## Question 1

The hash given in the class is: **4748dfe624d759009e3486fd062cd21dcf1858be435ea51c7ec7b4e53f52e02e**

We will hash the file with the following command:

```
openssl sha256 slides_w3.pdf
```

The output given was: **SHA2-256(slides_w5.pdf)= 4748dfe624d759009e3486fd062cd21dcf1858be435ea51c7ec7b4e53f52e02e**

In fact, it was returned the same hash.

### Question 1.1

It can be concluded that the integrity of the file hasn't been affected at all. The file has not been modified in any possible way.

### Question 1.2

Assuming it is asked, how many bytes **of the hash** will be affected with this change, the answer would be all of them. Every byte (or at least the most of them) are expected to change, giving an absolutely different output.

Let's prove this.

It's been changed 4 bytes of the file with help of the **Hex Editor** VSCode Extension by Microsoft.

```
openssl sha256 answer/slides_w5_modified.pdf 
```

The output given was: **SHA2-256(answer/slides_w5_modified.pdf)= 3eb92ec478789b68d623f7e7915a7eb3abb84487bcf7edf822b237c18ba0e3bb**
We can appreciate that it is absolutely different.

## Question 2

### Question 2.1

The answers are in the **crack_hash.py** file, below the *Exercise 1* comment as intended by the assignment.

### Question 2.2

To do this analysis, I first need to calculate the time it would actually need (in average) for each interaction when trying to find the respective hash.

Let's use the sane function I implemented for answering question 2.1, but just with some timers to see how much it lasts.

Let's first see how much is the rate for calculating hashes per second that my machine has for **SHA256** particularly.

In [18]:
import time, hashlib, os

def sha256_rate(trials=1_000_000, pwlen=8, saltlen=8):
    pwd = os.urandom(pwlen); salt = os.urandom(saltlen)
    t0 = time.perf_counter()
    for _ in range(trials):
        hashlib.sha256(salt + pwd).digest()
    return trials / (time.perf_counter() - t0)

R = sha256_rate()
print(f"{R:,.0f} hashes/s")

3,029,673 hashes/s


In [19]:
from crack_hash import hex_passwds, hlist, salt_passwds, shlist, mixed_hlist, mixed_shlist

def crack_time(D, N, R, salt_bytes=0, same_salt=True):
    # Returns worst-case time to cover the needed work (seconds)
    if salt_bytes == 0:
        return D / R                      # unsalted: one pass finds all matches
    if same_salt:
        return (2**(8*salt_bytes)) * D / R    # same salt for all hashes. 2^8 tries per byte. For n bytes, 2^(8*n)
    # different salt per hash
    return (2**(8*salt_bytes)) * D * N / R    # this case we gotta multiply it for N as well (number of hashes to find)

# Example usage (fill D, N from your data)
D = len(hex_passwds)   # or your keyspace size
N = len(hlist)




- **Cracking this set of passwords without salt**

In [20]:
print(f"Unsalted: {crack_time(D, N, R)} s")

# We can admit that this is virtually instantaneous. It is 6 orders of magnitude less than 1 second approximately.

Unsalted: 6.6013726199889795e-06 s


- **Cracking this set of passwords with the same 1 byte salt, where all hashes use the same salt**

In [21]:
print("Same 1-byte salt (known):", crack_time(D, N, R, salt_bytes=1, same_salt=True), "s")

# It is a lot longer that the previous one, but still very small (around 0.002s in my machine).

Same 1-byte salt (known): 0.0016899513907171787 s


- **Cracking this set of passwords with the same 8 byte salt, where all hashes use the same salt**

In [22]:
time_taken = crack_time(D, N, R, salt_bytes=8, same_salt=True)
print("Same 8-byte salt (known):", time_taken, "s")

time_days = time_taken / (60*60*24)
print(f"Which is approximately: {time_days:.2f} days")

time_years = time_days / 365
print(f"Or approximately: {time_years:.2f} years")

time__milleniums = time_years / 1000
print(f"Or approximately: {time__milleniums:.2f} milleniums")

#This is absolutely impractical. We are talking about multiple years to try all combinations in the worst case.

Same 8-byte salt (known): 121773831256130.2 s
Which is approximately: 1409419343.24 days
Or approximately: 3861422.86 years
Or approximately: 3861.42 milleniums


- **Cracking this set of passwords with the same 8 byte salt, where all hashes use different salts**

In [23]:
time = crack_time(D, N, R, salt_bytes=8, same_salt=False)
print("8-byte salt, different per hash (known):", time, "s")
time_years = time / (60*60*24*365)
print(f"Which is approximately: {time_years:.2f} years")
time__milleniums = time_years / 1000
print(f"Or approximately: {time__milleniums:.2f} milleniums")
#This is even more impractical. We are talking about multiple milleniums to try all combinations in the worst case.


8-byte salt, different per hash (known): 2435476625122604.0 s
Which is approximately: 77228457.16 years
Or approximately: 77228.46 milleniums


## Question 3

Following the steps given in this respective question. I downloaded 2 images .jpeg images with less than 64kb of space. Proceeded to use the tool and generated two pdf files. These files are said to have the same hash value for **SHA-1**. We'll test this with **OpenSSL**

Let's execute the following commands (you need to be located in the /images_q3/sha/ subdirectory).

For the first file:

```bash
openssl sha1 a.pdf

SHA1(a.pdf)= 2c64bc231817a1834ed4c371d44191a24a3d57f3
```
For the second file:

```bash
openssl sha1 b.pdf

SHA1(b.pdf)= 2c64bc231817a1834ed4c371d44191a24a3d57f3
```

So, in fact, these PDFs have the exact same hash value for SHA-1 function.


Now, let's get an overview about the **SHA-1 collision attack** paper by the SHAttered group.

First, in a nutshell, this attack is based on (for 3 colliding messages) to find/generate a difference in the second block pair that cancels out the difference initially caused by the first block. This leads again to identical chaining values and hence a collision.

It is employed *differential paths* to describe precisely the differences in state words **and** message words and of how these differences should propagate through the **80 steps** of the **SHA-1 hash function**.

This is done by modifying message words and thus directly influencing the next 16 state words (have in mind that the first 5 states are fixes by the **chaining value**).

The key for the solution to this, is the concept of *local collisions*, where any state bit difference introduced is to be canceled in the next 5 steps using correction message bit difference.

To ensure evetr message word but differences are compatible with the linea message expansion though, it is used a *Disturbance Vector (DV)*, that, to sum it up, it is a correctly expanded message itself, but where every "1" bit marks the start of a **local collision**.

It's utterly important the selection of a good disturbance vector. That's why the main reason of using two blocks pair instead of one, this choice alleviates a restriction of this vector itself which is that there are no state differences after the last step.

Thanks to Wang et al, this was solved. Crafting the **tailored differential path** that over the first 16 steps connects the input chaining value differences to the local collision differences over the remaining steps.

So, it's necessary to choose a good disturbance vector, then for each near-collision attack craft a non-linear differential path, determine a system of equation over all steps and finally find a solution in the form if a near-collision message block pair. Important to know is that one can only craft the non-linear path for the second near-collision attack once the caining values resulting from the first block pair are known.